In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM,Input,Conv1D, SpatialDropout1D,Dense, BatchNormalization, Activation, MaxPooling1D, GlobalAveragePooling1D, Add,Flatten,Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import LearningRateScheduler,EarlyStopping

### Load Training Data

In [2]:
with open('Train_X.npy', 'rb') as f:
    Train_X = np.load(f,allow_pickle=True)

with open('Train_y.npy', 'rb') as f:
    Train_y = np.load(f,allow_pickle=True)

with open('Test_X.npy', 'rb') as f:
    Test_X = np.load(f,allow_pickle=True)
    
with open('Test_y.npy', 'rb') as f:
    Test_y = np.load(f,allow_pickle=True)

print(Train_X.shape,Train_y.shape,Test_X.shape,Test_y.shape)

(1600000, 512) (1600000,) (498, 512) (498,)


In [3]:
Train_y[Train_y == 4] = 1
Test_y[Test_y == 4] = 1

### Logsitic Regression

In [4]:
clf = LogisticRegression(random_state=0,solver='saga').fit(Train_X, Train_y)
clf.score(Test_X, Test_y)

0.5180722891566265

#### KNN

In [5]:
neigh = KNeighborsClassifier(n_neighbors=11)
neigh.fit(Train_X, Train_y)
neigh.score(Test_X, Test_y)

0.5160642570281124

#### CNN

In [6]:
def get_model():
    model = Sequential()
    model.add(Input((512,1)))

    model.add(Conv1D(32,kernel_size=5,activation='relu'))
    model.add(MaxPooling1D())

    model.add(Conv1D(32,kernel_size=5,activation='relu'))
    model.add(MaxPooling1D())

    model.add(GlobalAveragePooling1D())
   
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

def step_decay(epoch):
    """
    a function to decay the learning rate 0.94 every 2 epoch
    """
    initial_lrate = 0.01
    drop = 0.95
    epochs_drop = 2
    end_lr = 0.00001
    lrate = initial_lrate * np.power(drop,  
        np.floor((1+epoch)/epochs_drop))
    if lrate > end_lr:
        return lrate
    else:
        return end_lr
lr_scheduler = LearningRateScheduler(step_decay)
el = EarlyStopping(monitor='val_loss', patience=8)

#parameter setting
epochs = 10
batch_size = 128
optimizer = Adam(0.01)

#model compile
model = get_model()
model.compile(optimizer,loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(np.expand_dims(Train_X,axis=2),Train_y,epochs = epochs, 
                                      batch_size = batch_size,validation_data=(np.expand_dims(Test_X,axis=2),Test_y),
                                      callbacks=[lr_scheduler,el],verbose = 1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 508, 32)           192       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 254, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 250, 32)           5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 125, 32)           0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9

2021-12-12 21:53:02.186610: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-12 21:53:02.186881: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12498/12500 [============================>.] - ETA: 0s - loss: 0.6903 - accuracy: 0.5300WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x17ab8b4c0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12500/12500 [==============================] - 321s 26ms/step - loss: 0.6903 - accuracy: 0.5300 - val_loss: 0.6319 - val_accuracy: 0.3795
Epoch 2/10
12500/12500 [==============================] - 317s 25ms/step - loss: 0.6891 - accuracy: 0.5362 - val_loss: 0.6694 - val_accuracy: 0.3996
Epoch 3/10
12500/12500 [==============================] - 320s 26ms/step - loss: 0.6885 - accuracy: 0.5385 - val_loss: 0.6682 - val_accuracy: 0.3855
Epoch 4/10
12500/12500 [==============================] - 309s 25ms/step - loss: 0.6882 - accuracy: 0.5392 - val_loss: 0.6513 - val_accuracy: 0.3735
Epoch 5/10
12500/12500 [==============================] - 300s 24ms/step - loss: 0.6880 - accuracy